In this tutorial we will learn to:
- Instantiate a DeepPrintExtractor
- Prepare a training dataset
- Train a DeepPrintExtractor

## Instantiate a DeepPrintExtractor

This package implements a number of variants of the DeepPrint architecture. The wrapper class for all these variants is called `DeepPrintExtractor`.
It has a `fit` method to train (and save) the model as well as an `extract` method to extract the DeepPrint features for fingerprint images. 

You can also try to implement your own models, but currently this is not directly supported by the package.

In [1]:
from flx.data.dataset import IdentifierSet, Identifier
from flx.extractor.fixed_length_extractor import get_DeepPrint_TexMinu, DeepPrintExtractor
from flx.benchmarks.verification import VerificationBenchmark
from flx.benchmarks.biometric_comparison import BiometricComparison

# We will use the example dataset with 10 subjects and 10 impressions per subject
# Split into 5 training subjects and 5 validation subjects
TRAINING_SUBJECTS = list(range(5))       # Subjects 0-4 for training
VALIDATION_SUBJECTS = list(range(5, 10)) # Subjects 5-9 for validation
IMPRESSIONS_PER_SUBJECT = list(range(10))

training_ids: IdentifierSet = IdentifierSet([
    Identifier(s, i) for s in TRAINING_SUBJECTS for i in IMPRESSIONS_PER_SUBJECT
])
validation_ids: IdentifierSet = IdentifierSet([
    Identifier(s, i) for s in VALIDATION_SUBJECTS for i in IMPRESSIONS_PER_SUBJECT
])

print(f"Training: {training_ids.num_subjects} subjects, {len(training_ids)} samples")
print(f"Validation: {validation_ids.num_subjects} subjects, {len(validation_ids)} samples")

# Create the extractor (note: num_training_subjects must match training set)
extractor: DeepPrintExtractor = get_DeepPrint_TexMinu(
    num_training_subjects=training_ids.num_subjects, 
    num_dims=256
)

# Create a simple verification benchmark for our small validation set
def create_simple_benchmark(subjects, impressions):
    """Create mated and non-mated comparisons for small datasets."""
    comparisons = []
    
    # Mated comparisons: same subject, different impressions
    for s in subjects:
        for i, imp1 in enumerate(impressions):
            for imp2 in impressions[i+1:]:
                comparisons.append(BiometricComparison(
                    Identifier(s, imp1), Identifier(s, imp2)
                ))
    
    # Non-mated comparisons: different subjects, same impression
    for i, s1 in enumerate(subjects):
        for s2 in subjects[i+1:]:
            for imp in impressions[:2]:  # Use first 2 impressions for non-mated
                comparisons.append(BiometricComparison(
                    Identifier(s1, imp), Identifier(s2, imp)
                ))
    
    return VerificationBenchmark(comparisons)

validation_benchmark = create_simple_benchmark(VALIDATION_SUBJECTS, IMPRESSIONS_PER_SUBJECT)
print(f"Validation benchmark: {len(validation_benchmark._comparisons)} comparisons")

Created IdentifierSet with 5 subjects and a total of 50 samples.
Created IdentifierSet with 5 subjects and a total of 50 samples.
Training: 5 subjects, 50 samples
Validation: 5 subjects, 50 samples
Validation benchmark: 245 comparisons


## Training the model

Instantiating the model was easy. To train it, first we will load the training data (see the [data tutorial](./dataset_tutorial.ipynb) for how to implement your own dataset).

Besides the fingerprint images, we also need a mapping from subjects to integer labels (for pytorch). For some variants we also need minutiae data. To see how a more complex dataset can be loaded, have a look at `flx/setup/datasets.py`.

Finally, we call the `fit` method, which trains the model and saves it to the specified path.

### Validation

We use a validation set to compute the **Equal Error Rate (EER)** during training. This is a proper biometric metric that measures how well the model distinguishes between same-subject (mated) and different-subject (non-mated) fingerprint pairs.

The validation benchmark defines which pairs to compare:
- **Mated comparisons**: Same subject, different impressions (should have high similarity)
- **Non-mated comparisons**: Different subjects (should have low similarity)

In [2]:
import os

import torch 

from flx.data.dataset import *
from flx.data.image_loader import SFingeLoader
from flx.data.minutia_map_loader import SFingeMinutiaMapLoader
from flx.data.label_index import LabelIndex
from flx.data.transformed_image_loader import TransformedImageLoader
from flx.image_processing.binarization import LazilyAllocatedBinarizer
from flx.data.image_helpers import pad_and_resize_to_deepprint_input_size

# NOTE: If this does not work, enter the absolute paths manually here! 
DATASET_DIR: str = os.path.abspath("/workspaces/fixed-length-fingerprint-extractors/notebooks/example-dataset")
MODEL_OUTDIR: str = os.path.abspath("/workspaces/fixed-length-fingerprint-extractors/example-model")

# We will use the SFingeLoader to load the images from the dataset
image_loader = TransformedImageLoader(
        images=SFingeLoader(DATASET_DIR),
        poses=None,
        transforms=[
            LazilyAllocatedBinarizer(5.0),
            pad_and_resize_to_deepprint_input_size,
        ],
    )

# Training datasets
image_dataset = Dataset(image_loader, training_ids)
label_dataset = Dataset(LabelIndex(training_ids), training_ids)
minutia_maps_dataset = Dataset(SFingeMinutiaMapLoader(DATASET_DIR), training_ids)

# Validation dataset (uses same image_loader, different identifiers)
validation_fingerprints = Dataset(image_loader, validation_ids)

print("=" * 60)
print("Training TexMinu")
print("=" * 60)

extractor.fit(
    fingerprints=image_dataset,
    minutia_maps=minutia_maps_dataset,
    labels=label_dataset,
    validation_fingerprints=validation_fingerprints,
    validation_benchmark=validation_benchmark,
    num_epochs=20,
    out_dir=MODEL_OUTDIR
)

print("\nTexMinu training complete!")
print(f"Model saved to: {MODEL_OUTDIR}")

Created IdentifierSet with 10 subjects and a total of 100 samples.
Created IdentifierSet with 10 subjects and a total of 100 samples.
Training TexMinu
Using device cuda:0
Loaded existing model from /workspaces/fixed-length-fingerprint-extractors/example-model/model.pyt


 --- Starting Epoch 1 of 20 ---

Training:


  0%|          | 0/4 [00:00<?, ?it/s]/workspaces/fixed-length-fingerprint-extractors/flx/models/model_training.py:118: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  epoch_loss += float(loss) * fp_labels.shape[0]
100%|██████████| 4/4 [00:04<00:00,  1.05s/it]


Average Loss: 1943821.2834826738
Multiclass accuracy: 0.20000000298023224

Validation:


100%|██████████| 2/2 [00:03<00:00,  1.87s/it]


Created IdentifierSet with 5 subjects and a total of 50 samples.
Created IdentifierSet with 5 subjects and a total of 50 samples.
Created IdentifierSet with 5 subjects and a total of 50 samples.


...

Average Loss: 495.17008304405215
Multiclass accuracy: 0.5799999833106995

Validation:


100%|██████████| 2/2 [00:03<00:00,  1.83s/it]


Created IdentifierSet with 5 subjects and a total of 50 samples.
Created IdentifierSet with 5 subjects and a total of 50 samples.
Created IdentifierSet with 5 subjects and a total of 50 samples.


100%|██████████| 245/245 [00:00<00:00, 148155.20it/s]

Equal Error Rate: 0.2

TrainingLogEntry(
    epoch=20,
    training_loss=495.17008304405215,
    loss_statistics={'texture_loss': {'crossent_loss_sum': 0.004459361016750336, 'center_loss_sum': 0.00023008433310315013}, 'minutia_loss': {'crossent_loss_sum': 0.005879925727844238, 'center_loss_sum': 0.00018789409473538398}, 'minutia_map_loss': 1.846053861808777},
    training_accuracy=0.5799999833106995,
    validation_equal_error_rate=0.2,
}

TexMinu training complete!
Model saved to: /workspaces/fixed-length-fingerprint-extractors/example-model


## Training with Localization Network (LocTexMinu)

The **LocTexMinu** variant adds a **Localization Network** that learns to automatically align fingerprints before feature extraction. This is useful when input images may have rotation or translation variations.

| Variant | Texture Branch | Minutia Branch | Localization Network |
|---------|----------------|----------------|----------------------|
| TexMinu | ✅ | ✅ | ❌ |
| LocTexMinu | ✅ | ✅ | ✅ |

The localization network uses a spatial transformer to predict and apply an affine transformation, making the model more robust to pose variations in the input.

In [3]:
from flx.extractor.fixed_length_extractor import get_DeepPrint_LocTexMinu

print("=" * 60)
print("Training LocTexMinu (with Localization Network)")
print("=" * 60)

# LocTexMinu adds a localization network for learned alignment
extractor_loc: DeepPrintExtractor = get_DeepPrint_LocTexMinu(
    num_training_subjects=training_ids.num_subjects, 
    num_dims=256
)

# Use a separate output directory to avoid overwriting the TexMinu model
MODEL_OUTDIR_LOC: str = os.path.abspath("/workspaces/fixed-length-fingerprint-extractors/example-model-loctexminu")

# Training uses the same datasets as TexMinu - including validation
extractor_loc.fit(
    fingerprints=image_dataset,
    minutia_maps=minutia_maps_dataset,
    labels=label_dataset,
    validation_fingerprints=validation_fingerprints,
    validation_benchmark=validation_benchmark,
    num_epochs=20,
    out_dir=MODEL_OUTDIR_LOC
)

print("\nLocTexMinu training complete!")
print(f"Model saved to: {MODEL_OUTDIR_LOC}")

Training LocTexMinu (with Localization Network)
Using device cuda:0
No model file found at /workspaces/fixed-length-fingerprint-extractors/example-model-loctexminu/model.pyt


 --- Starting Epoch 1 of 20 ---

Training:


100%|██████████| 4/4 [00:02<00:00,  1.49it/s]


Average Loss: 159347.40091804496
Multiclass accuracy: 0.20000000298023224

Validation:


100%|██████████| 2/2 [00:03<00:00,  1.84s/it]


Created IdentifierSet with 5 subjects and a total of 50 samples.
Created IdentifierSet with 5 subjects and a total of 50 samples.
Created IdentifierSet with 5 subjects and a total of 50 samples.


...

Average Loss: 221.17291870474816
Multiclass accuracy: 0.17999999225139618

Validation:


100%|██████████| 2/2 [00:03<00:00,  1.86s/it]


Created IdentifierSet with 5 subjects and a total of 50 samples.
Created IdentifierSet with 5 subjects and a total of 50 samples.
Created IdentifierSet with 5 subjects and a total of 50 samples.


100%|██████████| 245/245 [00:00<00:00, 150520.65it/s]


Equal Error Rate: 0.4

TrainingLogEntry(
    epoch=20,
    training_loss=221.17291870474816,
    loss_statistics={'texture_loss': {'crossent_loss_sum': 0.006816561102867127, 'center_loss_sum': 0.00016852886322885752}, 'minutia_loss': {'crossent_loss_sum': 0.006615631818771363, 'center_loss_sum': 0.00015070945769548417}, 'minutia_map_loss': 0.9910800753884016},
    training_accuracy=0.17999999225139618,
    validation_equal_error_rate=0.4,
}

LocTexMinu training complete!
Model saved to: /workspaces/fixed-length-fingerprint-extractors/example-model-loctexminu


In [4]:
import torch
from flx.models.torch_helpers import get_device

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Current device: {get_device()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

CUDA available: True
Current device: cuda:0
GPU name: NVIDIA GeForce RTX 5090
